In [1]:
# ==================== OPTIMIZATION FLAGS ====================
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_GPU_THREAD_COUNT'] = '2'

In [2]:
%%writefile env.py

"""
Waffle Game Environment - optimized for speed
"""
import numpy as np
import json
from typing import Tuple, List, Optional, Sequence

class Waffle(object):
    def __init__(self, puzzles_file: str = "waffles_shuffled.jsonl",
                 embeddings_file: str = "letter_embeddings.npy") -> None:
        self.puzzles_file = puzzles_file
        self.puzzles = self._load_puzzles()
        self.embeddings = self._load_embeddings(embeddings_file)
        self.grid_size = 21

        self.row_words = [
            [0, 1, 2, 3, 4],
            [8, 9, 10, 11, 12],
            [16, 17, 18, 19, 20]
        ]
        self.col_words = [
            [0, 5, 8, 13, 16],
            [2, 6, 10, 14, 18],
            [4, 7, 12, 15, 20]
        ]

        # Pre-allocate arrays
        self.current_state = np.zeros(21, dtype=np.int32)
        self.target_state = np.zeros(21, dtype=np.int32)

        # features: [green, yellow, emb(8)] => shape (21,10)
        self.prev_features = np.zeros((21, 10), dtype=np.float32)
        self.features_buffer = np.zeros((21, 10), dtype=np.float32)

        # Precompute position->word mapping as numpy arrays for faster access
        self.pos_to_word_positions = [None] * 21
        for i in range(21):
            wp = None
            for word in self.row_words:
                if i in word:
                    wp = word
                    break
            if wp is None:
                for word in self.col_words:
                    if i in word:
                        wp = word
                        break
            self.pos_to_word_positions[i] = np.array(wp, dtype=np.int32) if wp else None

        # will be filled in reset
        self.reset(0)

    def _load_puzzles(self) -> List[dict]:
        puzzles = []
        with open(self.puzzles_file, "r") as f:
            for line in f:
                puzzles.append(json.loads(line))
        return puzzles

    def _load_embeddings(self, embeddings_file: str) -> np.ndarray:
        try:
            embeddings = np.load(embeddings_file)
            return embeddings.astype(np.float32)
        except FileNotFoundError:
            return np.random.randn(27, 8).astype(np.float32)

    def _string_to_array(self, flat21: str, out: np.ndarray) -> None:
        # faster vectorized conversion
        arr = np.frombuffer(flat21.encode('ascii'), dtype=np.uint8) - ord('a') + 1
        out[:] = arr.astype(np.int32)

    def _calculate_features_full(self, state: np.ndarray, out: np.ndarray) -> None:
        """Compute features for all positions (vectorized)."""
        # green mask
        green = (state == self.target_state).astype(np.float32)
        out[:, 0] = green

        # yellow: for each pos, check if current_letter matches any target letter in its word (excluding green)
        for pos in range(21):
            wp = self.pos_to_word_positions[pos]
            if wp is None:
                out[pos, 1] = 0.0
                continue
            current_letter = state[pos]
            # check if current_letter equals target_state at any wp position (and not already green at pos)
            matches = (self.target_state[wp] == current_letter)
            out[pos, 1] = 1.0 if matches.any() and not green[pos] else 0.0

        # embeddings
        # index clipping
        indices = np.clip(state.astype(np.int32), 0, 26)
        out[:, 2:10] = self.embeddings[indices]

    def _calculate_features_positions(self, state: np.ndarray, out: np.ndarray, positions: Sequence[int]) -> None:
        """Only update provided positions in 'out' using same logic as full."""
        for pos in positions:
            current_letter = state[pos]
            out[pos, :] = 0.0
            # green
            if current_letter == self.target_state[pos]:
                out[pos, 0] = 1.0
            else:
                wp = self.pos_to_word_positions[pos]
                if wp is not None:
                    if np.any(self.target_state[wp] == current_letter):
                        out[pos, 1] = 1.0
            # embeddings
            idx = int(current_letter) if current_letter <= 26 else 0
            out[pos, 2:10] = self.embeddings[idx]

    def _count_correct_positions(self, state: np.ndarray) -> int:
        return int(np.sum(state == self.target_state))

    def reset(self, index: int) -> None:
        self.current_puzzle = self.puzzles[index]
        self._string_to_array(self.current_puzzle["target_flat21"], self.target_state)
        self._string_to_array(self.current_puzzle["shuffled_flat21"], self.current_state)
        self.fixed_indices = set(self.current_puzzle["fixed_indices"])
        self.moves = 0
        self.max_moves = 200
        self.prev_correct_count = self._count_correct_positions(self.current_state)
        # Precompute movable positions & valid actions once per puzzle
        self.movable_positions = np.array([i for i in range(21) if i not in self.fixed_indices], dtype=np.int32)
        # precompute valid actions (pos1,pos2) encoded as pos1*21+pos2 for pos1 < pos2
        valid_actions = []
        mp = self.movable_positions
        for i in range(len(mp)):
            for j in range(i+1, len(mp)):
                valid_actions.append(int(mp[i] * 21 + mp[j]))
        self._valid_actions_cache = np.array(valid_actions, dtype=np.int32)
        # full features at start
        self._calculate_features_full(self.current_state, self.prev_features)
        self.features_buffer[:] = self.prev_features

    def _update_state(self, action: int) -> None:
        pos1 = action // 21
        pos2 = action % 21
        if pos1 not in self.fixed_indices and pos2 not in self.fixed_indices:
            self.current_state[pos1], self.current_state[pos2] = self.current_state[pos2], self.current_state[pos1]
        self.moves += 1

    def _get_reward(self, action: int) -> float:
        pos1 = action // 21
        pos2 = action % 21
        reward = -0.1

        if np.array_equal(self.current_state, self.target_state):
            return 1000.0 + reward

        current_correct = self._count_correct_positions(self.current_state)
        correct_change = current_correct - self.prev_correct_count

        # Only recompute features for the two positions that changed (big speed win).
        self._calculate_features_positions(self.current_state, self.features_buffer, [pos1, pos2])

        if self.current_state[pos1] == self.current_state[pos2]:
            reward += -1

        if (self.current_state[pos2] == self.target_state[pos1] or
            self.current_state[pos1] == self.target_state[pos2]):
            reward += -10

        if correct_change > 0:
            reward += 1 * int(correct_change)
        elif correct_change < 0:
            reward += -2 * abs(int(correct_change))

        # feature-change bonuses (only check pos1,pos2)
        for pos in (pos1, pos2):
            prev_green = self.prev_features[pos, 0]
            prev_yellow = self.prev_features[pos, 1]
            curr_green = self.features_buffer[pos, 0]
            curr_yellow = self.features_buffer[pos, 1]

            if prev_green == 0 and prev_yellow == 0 and curr_yellow == 1.0:
                reward += 1
            elif prev_yellow == 1.0 and curr_green == 0 and curr_yellow == 0:
                reward += -1

        self.prev_correct_count = current_correct
        # copy updated positions back into prev_features (only the changed positions)
        self.prev_features[[pos1, pos2], :] = self.features_buffer[[pos1, pos2], :]
        return float(reward)

    def _is_over(self) -> bool:
        return np.array_equal(self.current_state, self.target_state) or self.moves >= self.max_moves

    def _is_win(self) -> bool:
        return np.array_equal(self.current_state, self.target_state)

    def observe(self) -> np.ndarray:
        # returns a view shaped for model
        return self.prev_features.reshape(1, 21, 10)

    def act(self, action: int) -> Tuple[np.ndarray, float, bool, bool]:
        self._update_state(action)
        reward = self._get_reward(action)
        game_over = self._is_over()
        game_win = self._is_win()
        return self.observe(), reward, game_over, game_win

    def get_valid_actions(self) -> List[int]:
        # return cached list (fast)
        return self._valid_actions_cache.tolist()

    def get_num_actions(self) -> int:
        return 21 * 21

Writing env.py


In [3]:
%%writefile experience_replay.py

# experience_replay.py (corrected)
"""
Experience Replay - circular buffer based for speed (corrected)
"""
import numpy as np
import tensorflow as tf
from typing import Tuple

class ExperienceReplay(object):
    def __init__(self, max_memory: int = 5000, discount: float = 0.95) -> None:
        self.max_memory = int(max_memory)
        self.discount = float(discount)
        # We'll store states flattened for compact storage; each state originally shape (1,21,10)
        self.state_shape = (1, 21, 10)
        flat_size = int(np.prod(self.state_shape))
        self.states = np.zeros((self.max_memory, flat_size), dtype=np.float32)
        self.next_states = np.zeros((self.max_memory, flat_size), dtype=np.float32)
        self.actions = np.zeros((self.max_memory,), dtype=np.int32)
        self.rewards = np.zeros((self.max_memory,), dtype=np.float32)
        self.game_over = np.zeros((self.max_memory,), dtype=np.bool_)
        self._size = 0
        self._pos = 0

    def add_experience(self, sars: list, game_over: bool) -> None:
        # sars: [previous_state (1,21,10), action (int), reward (float), current_state (1,21,10)]
        prev_s, action, reward, curr_s = sars
        self.states[self._pos] = np.asarray(prev_s, dtype=np.float32).reshape(-1)
        self.next_states[self._pos] = np.asarray(curr_s, dtype=np.float32).reshape(-1)
        self.actions[self._pos] = int(action)
        self.rewards[self._pos] = float(reward)
        self.game_over[self._pos] = bool(game_over)
        self._pos = (self._pos + 1) % self.max_memory
        self._size = min(self._size + 1, self.max_memory)

    def size(self) -> int:
        """Public accessor for number of stored transitions."""
        return int(self._size)

    def get_qlearning_batch(self, model, batch_size: int = 32) -> Tuple[np.ndarray, np.ndarray]:
        if self._size == 0:
            raise ValueError("No experience to sample")
        num_samples = min(self._size, int(batch_size))
        ids = np.random.choice(self._size, size=num_samples, replace=False)

        # restore to (batch, 21, 10)
        previous_states = self.states[ids].reshape((num_samples, 21, 10)).astype(np.float32)
        current_states = self.next_states[ids].reshape((num_samples, 21, 10)).astype(np.float32)
        action_ts = self.actions[ids].astype(np.int32)
        rewards = self.rewards[ids].astype(np.float32)
        game_over = self.game_over[ids]

        # Ensure inputs are tf tensors with correct dtype/device
        prev_tf = tf.constant(previous_states, dtype=tf.float32)
        next_tf = tf.constant(current_states, dtype=tf.float32)

        # Model inference (dtype stable)
        targets = model(prev_tf, training=False).numpy()
        Q_next = model(next_tf, training=False).numpy()
        Q_sa = Q_next.max(axis=1)

        # compute RHS then cast to targets' dtype
        not_done = (~game_over).astype(np.float32)
        rhs = (rewards + self.discount * Q_sa * not_done).astype(targets.dtype)

        targets[np.arange(num_samples), action_ts] = rhs

        return previous_states, targets

Writing experience_replay.py


In [4]:
# ==================== CELL 3: Main Training ====================
import tensorflow as tf
print(f"TensorFlow: {tf.__version__}")
print(f"GPUs: {len(tf.config.list_physical_devices('GPU'))}")

import json
import numpy as np
import logging
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

from experience_replay import ExperienceReplay
from env import Waffle

# ==================== GPU CONFIGURATION ====================
print("="*60)
print("GPU OPTIMIZATION")
print("="*60)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Critical optimizations
        tf.config.threading.set_intra_op_parallelism_threads(8)
        tf.config.threading.set_inter_op_parallelism_threads(4)
        tf.config.optimizer.set_jit(True)
        
        print(f"✓ {len(gpus)} GPU(s) configured")
        print("✓ XLA compilation enabled")
        print("✓ Thread optimization enabled")
    except RuntimeError as e:
        print(f"Warning: {e}")
else:
    print("⚠️ No GPU")
print("="*60)

import tensorflow as tf
from tensorflow.keras import mixed_precision

# --- Enable mixed precision for speed (if GPU supports it) ---
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print("Mixed precision policy:", mixed_precision.global_policy())

# Keep XLA enabled (you already set jit via compile and optimizer options)
tf.config.optimizer.set_jit(True)

# Reduce Python prints in the hot loop (optional)
# Comment out model.summary() if you want speed during startup

# ==================== FIND DATASET ====================
dataset_path = None
embeddings_path = None

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename == 'waffles_shuffled.jsonl':
            dataset_path = os.path.join(dirname, filename)
        if filename == 'letter_embeddings.npy':
            embeddings_path = os.path.join(dirname, filename)

if not dataset_path:
    raise FileNotFoundError("waffles_shuffled.jsonl not found")

print(f"\n✓ Dataset: {dataset_path}")
print(f"✓ Embeddings: {embeddings_path}\n")

# ==================== MODEL DEFINITION ====================
def define_model(input_shape, hidden_size, num_actions, learning_rate=0.001, conv_filters=64):
    model = Sequential([
        Conv1D(conv_filters, 3, activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        Conv1D(conv_filters, 3, activation='relu', padding='same'),
        BatchNormalization(),
        Conv1D(conv_filters * 2, 5, activation='relu', padding='same'),
        BatchNormalization(),
        Flatten(),
        Dense(hidden_size, activation='relu'),
        Dropout(0.2),
        Dense(hidden_size // 2, activation='relu'),
        Dropout(0.2),
        Dense(num_actions)
    ])
    
    from tensorflow.keras import mixed_precision
    # Create base optimizer
    base_opt = Adam(learning_rate=learning_rate)
    # Wrap into LossScaleOptimizer if available (Keras mixed precision)
    try:
        opt = mixed_precision.LossScaleOptimizer(base_opt)
    except Exception:
        # fallback to base optimizer if wrapper not available
        opt = base_opt

    model.compile(
        optimizer=opt,
        loss='mse',
        jit_compile=True
    )

    return model

# ==================== TRAINING FUNCTION ====================

@tf.function(reduce_retracing=True)
def train_step(model, inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        # compute loss in float32 consistently
        loss = tf.reduce_mean(tf.square(tf.cast(predictions, tf.float32) - tf.cast(targets, tf.float32)))

    optimizer = model.optimizer

    # If optimizer supports Keras LossScaleOptimizer API, use it; else fallback to manual scaling.
    if hasattr(optimizer, "get_scaled_loss") and hasattr(optimizer, "get_unscaled_gradients"):
        scaled_loss = optimizer.get_scaled_loss(loss)
        scaled_grads = tape.gradient(scaled_loss, model.trainable_variables)
        grads = optimizer.get_unscaled_gradients(scaled_grads)
    else:
        # manual loss-scaling fallback
        loss_scale = tf.constant(1024.0, dtype=tf.float32)
        scaled_loss = loss * loss_scale
        scaled_grads = tape.gradient(scaled_loss, model.trainable_variables)
        # unscale grads (guard against None)
        grads = [g / loss_scale if g is not None else tf.zeros_like(v) for g, v in zip(scaled_grads, model.trainable_variables)]

    # Replace any remaining None gradients with zeros
    grads = [g if g is not None else tf.zeros_like(v) for g, v in zip(grads, model.trainable_variables)]

    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

def train_model(model, env, epochs, experience_replay, 
               epsilon_start, epsilon_end, epsilon_decay, batch_size):
    
    win_count = 0
    total_rewards = []
    epsilon = epsilon_start
    num_actions = env.get_num_actions()
    num_puzzles = len(env.puzzles)
    
    # Pre-compile prediction function
    @tf.function
    def predict_q(state):
        return model(state, training=False)
    
    print("\n🚀 Starting training...\n")
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        batch_count = 0
        
        for i in range(num_puzzles):
            env.reset(i)
            current_state = env.observe()
            game_over = False
            episode_reward = 0
            step_count = 0
            
            while not game_over:
                previous_state = current_state
                
                # Action selection
                if np.random.rand() <= epsilon:
                    valid_actions = env.get_valid_actions()
                    action = np.random.choice(valid_actions) if valid_actions else np.random.randint(0, num_actions)
                else:
                    q = predict_q(tf.constant(previous_state, dtype=tf.float32))
                    action = int(tf.argmax(q[0], axis=-1).numpy())
                
                current_state, reward, game_over, game_win = env.act(action)
                episode_reward += reward
                step_count += 1
                
                if game_win:
                    win_count += 1
                
                experience_replay.add_experience(
                    [previous_state, int(action), reward, current_state], game_over
                )
                
                # Batch training every 5 steps
                if experience_replay.size() >= batch_size and step_count % 5 == 0:
                    inputs, targets = experience_replay.get_qlearning_batch(model, batch_size)
                    inputs_tensor = tf.convert_to_tensor(inputs, dtype=tf.float32)
                    # ensure targets remain float32 for loss calculation (we cast preds to float32 inside train_step)
                    targets_tensor = tf.convert_to_tensor(targets, dtype=tf.float32)
                    loss = train_step(model, inputs_tensor, targets_tensor)
                    epoch_loss += loss.numpy()
                    batch_count += 1
            
            total_rewards.append(episode_reward)
        
        epsilon = max(epsilon_end, epsilon * epsilon_decay)
        
        # Progress reporting
        if (epoch + 1) % 10 == 0 or epoch < 10:
            avg_loss = epoch_loss / max(batch_count, 1)
            avg_reward = np.mean(total_rewards[-100:]) if len(total_rewards) >= 100 else np.mean(total_rewards)
            win_rate = win_count / ((epoch + 1) * num_puzzles) * 100
            
            print(f"Ep {epoch+1:4d}/{epochs} | Loss: {avg_loss:8.4f} | ε: {epsilon:.4f} | "
                  f"Wins: {win_count:4d} ({win_rate:5.1f}%) | AvgR: {avg_reward:7.2f}")
        
        if (epoch + 1) % 50 == 0:
            print(f"\n{'='*70}")
            print(f"CHECKPOINT - Epoch {epoch+1}")
            print(f"Win rate: {win_rate:.1f}% | Avg reward: {avg_reward:.2f}")
            print(f"{'='*70}\n")
    
    final_win_rate = win_count / (epochs * num_puzzles) * 100
    print(f"\n✅ Training complete! Win rate: {final_win_rate:.1f}%")
    return model

EPOCHS = 100
EPSILON_START = 1.0
EPSILON_END = 0.05
# Prefer per-step linear decay; if per-epoch multiplicative, use:
EPSILON_DECAY = 0.95

MAX_MEMORY = 20000
HIDDEN_SIZE = 256
CONV_FILTERS = 64
BATCH_SIZE = 128
DISCOUNT = 0.99
LEARNING_RATE = 3e-4

# ==================== INITIALIZE ====================
env = Waffle(puzzles_file=dataset_path, embeddings_file=embeddings_path)
num_actions = env.get_num_actions()
input_shape = (21, 10)

print(f"\n{'='*70}")
print(f"TRAINING CONFIGURATION")
print(f"{'='*70}")
print(f"Puzzles:             {len(env.puzzles)}")
print(f"Actions:             {num_actions}")
print(f"Input shape:         {input_shape}")
print(f"Batch size:          {BATCH_SIZE}")
print(f"Hidden size:         {HIDDEN_SIZE}")
print(f"Epochs:              {EPOCHS}")
print(f"{'='*70}\n")

# ==================== BUILD MODEL ====================
model = define_model(
    input_shape=input_shape,
    hidden_size=HIDDEN_SIZE,
    num_actions=num_actions,
    learning_rate=LEARNING_RATE,
    conv_filters=CONV_FILTERS
)

model.summary()

# GPU warmup
warmup_input = tf.random.normal([1, 21, 10])
_ = model(warmup_input, training=False)
print(f"\n✓ Model compiled with XLA")
print(f"✓ GPU warmup complete\n")

# ==================== TRAIN ====================
exp_replay = ExperienceReplay(max_memory=MAX_MEMORY, discount=DISCOUNT)

trained_model = train_model(
    model=model,
    env=env,
    epochs=EPOCHS,
    experience_replay=exp_replay,
    epsilon_start=EPSILON_START,
    epsilon_end=EPSILON_END,
    epsilon_decay=EPSILON_DECAY,
    batch_size=BATCH_SIZE
)

# ==================== SAVE ====================
output_dir = '/kaggle/working/model'
os.makedirs(output_dir, exist_ok=True)

trained_model.save_weights(f'{output_dir}/model.weights.h5')
with open(f'{output_dir}/model.json', 'w') as f:
    json.dump(trained_model.to_json(), f)

print(f"\n✅ Model saved to {output_dir}/")

2025-11-18 17:34:44.542299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763487284.735563      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763487284.791265      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

 Non-trainable params: 512 (2.00 KB)

I0000 00:00:1763487301.033807      48 cuda_dnn.cc:529] Loaded cuDNN version 90300



✓ Model compiled with XLA
✓ GPU warmup complete


🚀 Starting training...



I0000 00:00:1763487303.192169     110 service.cc:148] XLA service 0x7b251c006c80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763487303.192799     110 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1763487303.292627     110 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Ep    1/100 | Loss:   4.4716 | ε: 0.9500 | Wins:    0 (  0.0%) | AvgR: -553.46
Ep    2/100 | Loss:   0.2331 | ε: 0.9025 | Wins:    0 (  0.0%) | AvgR: -540.73
Ep    3/100 | Loss:   0.2318 | ε: 0.8574 | Wins:    0 (  0.0%) | AvgR: -524.11
Ep    4/100 | Loss:   0.2311 | ε: 0.8145 | Wins:    0 (  0.0%) | AvgR: -526.49
Ep    5/100 | Loss:   0.2311 | ε: 0.7738 | Wins:    0 (  0.0%) | AvgR: -530.68
Ep    6/100 | Loss:   0.2312 | ε: 0.7351 | Wins:    0 (  0.0%) | AvgR: -523.40
Ep    7/100 | Loss:   0.2322 | ε: 0.6983 | Wins:    0 (  0.0%) | AvgR: -512.32
Ep    8/100 | Loss:   0.2315 | ε: 0.6634 | Wins:    0 (  0.0%) | AvgR: -506.18
Ep    9/100 | Loss:   0.2297 | ε: 0.6302 | Wins:    0 (  0.0%) | AvgR: -500.98
Ep   10/100 | Loss:   0.2289 | ε: 0.5987 | Wins:    0 (  0.0%) | AvgR: -493.64
Ep   20/100 | Loss:   0.2266 | ε: 0.3585 | Wins:    0 (  0.0%) | AvgR: -457.64
Ep   30/100 | Loss:   0.2229 | ε: 0.2146 | Wins:    0 (  0.0%) | AvgR: -435.58
Ep   40/100 | Loss:   0.2244 | ε: 0.1285 | Wins:    